#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    concatenated_df = pd.concat([df,new_data])
    return concatenated_df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    print(data)
    return data
    

In [4]:
symbols = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]

Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).Close[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }
    print(input_data)
    data = load_and_setup_data(symbol,input_data)
    data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = DataProcessing(symbol,data)
    # Specify Target
    data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
    #final training
    loaded_models[symbol] = loaded_model
    data.df = data.df.dropna()

    final_target[symbol] = data.df["TARGET"]
    df = data.df[feature_item]
    finaldata[symbol]= df 
    stock_name_only = stock_name.replace(".NS","")
    train_yhat = loaded_model.predict(df[-1:])
    train_yhat_proba = loaded_model.predict_proba(df[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
#    print(greater)
    final_result[symbol] = [current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)])

print(result)


[*********************100%***********************]  8 of 8 completed
             Adj Close                                                   \
                BEL.NS   CONCOR.NS ELGIEQUIP.NS     IOC.NS        LT.NS   
Date                                                                      
2023-07-06  124.849998  689.849976        540.0  98.599998  2490.199951   

                                                         Close              \
             OLECTRA.NS TATAELXSI.NS         ^NSEI      BEL.NS   CONCOR.NS   
Date                                                                         
2023-07-06  1043.949951  7538.700195  19500.050781  124.849998  689.849976   

            ...         Open                  Volume                         \
            ... TATAELXSI.NS         ^NSEI    BEL.NS CONCOR.NS ELGIEQUIP.NS   
Date        ...                                                               
2023-07-06  ...  7566.149902  19385.699219  15544166   1195700       108842   



/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*

{'Open': [2468.10009765625], 'Close': [2490.199951171875], 'High': [2499.949951171875], 'Low': [2467.550048828125], 'Volume': [1553138], 'Date': ['2023-07-06']}


/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*

{'Open': [674.0], 'Close': [689.8499755859375], 'High': [691.5499877929688], 'Low': [671.0], 'Volume': [1195700], 'Date': ['2023-07-06']}


...

/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*i

{'Open': [7566.14990234375], 'Close': [7538.7001953125], 'High': [7566.89990234375], 'Low': [7510.0], 'Volume': [67227], 'Date': ['2023-07-06']}


/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*i

{'Open': [19385.69921875], 'Close': [19500.05078125], 'High': [19502.30078125], 'Low': [19373.0], 'Volume': [0], 'Date': ['2023-07-06']}
[['06-07-2023', 0, 1, 'xg', 'OLECTRA', 1043.949951171875, 88.93], ['06-07-2023', 0, 1, 'xg', 'LT', 2490.199951171875, 51.86], ['06-07-2023', 1, 1, 'xg', 'CONCOR', 689.8499755859375, 45.15], ['06-07-2023', 0, 1, 'xg', 'ELGIEQUIP', 540.0, 90.41], ['06-07-2023', 0, 1, 'xg', 'IOC', 98.5999984741211, 65.23], ['06-07-2023', 0, 1, 'xg', 'BEL', 124.8499984741211, 66.96], ['06-07-2023', 1, 1, 'xg', 'TATAELXSI', 7538.7001953125, 48.12], ['06-07-2023', 0, 1, 'xg', 'NSEI', 19500.05078125, 84.69]]


/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = df["Close"].pct_change()
/Users/vipin/workspace/ai/my_own/MY_OWN/LoadingNdRun/stratmanagerDaily.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Range"] = df["High"] / df["Low"] - 1
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*i

In [7]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '1043.949951171875',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'OLECTRA',
   'prob': '88.93'},
  {'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '2490.199951171875',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'LT',
   'prob': '51.86'},
  {'cdate': '06-07-2023',
   'result': '1',
   'pre_close': '689.8499755859375',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'CONCOR',
   'prob': '45.15'},
  {'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '540.0',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'ELGIEQUIP',
   'prob': '90.41'},
  {'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '98.5999984741211',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'IOC',
   'prob': '65.23'},
  {'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '124.8499984741211',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'BEL',
   'prob': '66.96'},
  {

In [8]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [9]:
final_target

{'OLECTRA.NS': Date
 2005-05-06    1.0
 2005-05-10    0.0
 2005-05-12    0.0
 2005-05-13    1.0
 2005-05-16    1.0
              ... 
 2023-06-27    0.0
 2023-06-28    1.0
 2023-06-30    0.0
 2023-07-03    0.0
 2023-07-05    0.0
 Name: TARGET, Length: 1818, dtype: float64,
 'LT.NS': Date
 2005-05-19    0.0
 2005-05-20    0.0
 2005-05-23    0.0
 2005-05-26    0.0
 2005-06-01    0.0
              ... 
 2023-06-22    0.0
 2023-06-27    0.0
 2023-06-28    1.0
 2023-06-30    0.0
 2023-07-04    1.0
 Name: TARGET, Length: 2289, dtype: float64,
 'CONCOR.NS': Date
 2005-05-12    0.0
 2005-05-13    0.0
 2005-05-16    0.0
 2005-05-18    0.0
 2005-05-19    0.0
              ... 
 2023-06-23    0.0
 2023-06-26    0.0
 2023-06-28    1.0
 2023-06-30    1.0
 2023-07-05    1.0
 Name: TARGET, Length: 2225, dtype: float64,
 'ELGIEQUIP.NS': Date
 2005-05-18    1.0
 2005-05-20    1.0
 2005-05-23    1.0
 2005-05-25    1.0
 2005-05-26    1.0
              ... 
 2023-06-14    1.0
 2023-06-16    0.0
 2023-06-2

In [10]:
for symbol in symbols:
    retrained_model = loaded_models[symbol].fit(finaldata[symbol],final_target[symbol])
    print(retrained_model)
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(retrained_model, f)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=6, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eva